In [1]:
#Imports
import sys
import os
import glob as glob
import pandas as pd
import numpy as np
from calvin_utils.file_utils.dataframe_utilities import preprocess_colnames_for_regression
np.set_printoptions(precision=3, suppress=True)

In [2]:
analysis = 'response_topology/voxelwise_mediated_moderation/age_mediated_by_grey_matter'
clin_path = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STN_WURZBURG/metadata/cog_decline_scores_simple.csv'
out_dir = os.path.join(os.path.dirname(clin_path), f'{analysis}')
save = True
if os.path.exists(out_dir):
    pass
else:
    os.makedirs(out_dir)

Define Data to Submit

In [3]:
#----------------------------------------------------------------user input above----------------------------------------------------------------
data_df = pd.read_csv(clin_path)
    
# #Remove subjects
# outlier_index=[11, 47, 48, 49]
# data_df = data_df.drop(index=outlier_index)
# data_df.reset_index(drop=True, inplace=True)

data_df

,ID,diff. Mattis,perc_change_mattis,perc_improvement,Age
0,1,8,5.673759,-5.673759,57.0
1,2,-3,-2.158273,2.158273,50.0
2,3,2,1.388889,-1.388889,62.0
3,4,3,2.142857,-2.142857,50.0
4,6,1,0.704225,-0.704225,60.0
5,7,1,0.709220,-0.709220,73.0
6,9,3,2.142857,-2.142857,64.0
7,11,1,0.694444,-0.694444,62.0
8,12,0,0.000000,0.000000,54.0
9,14,-1,-0.699301,0.699301,49.0


Outcome Data

In [4]:
subject_column = 'ID'
outcome_column = 'perc_improvement'
#----------------------------------------------------------------DO NOT MODIFY--------------------------------------------------------
outcomes_df = pd.DataFrame()
outcomes_df['outcome'] = data_df.loc[:, [outcome_column]]

#Standardize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
outcomes_df.iloc[:,:] = scaler.fit_transform(outcomes_df.iloc[:,:])


outcomes_df['subject_id'] = data_df.loc[:, [subject_column]]
# outcomes_df['subject_id'] = [id.split('_')[0] for id in data_df[subject_column].to_list()] #<------------------------problem code. must generalize to extract subject id better
outcomes_df.set_index('subject_id', inplace=True)

# data_df = data_df.set_index('Patient # CDR, ADAS')
# data_df['subject_id'] = data_df.index
# data_df['outcome'] = data_df

# Convert the 'subject_id' column to strings for each DataFrame
outcomes_df.index = outcomes_df.index.astype(str)

outcomes_df = preprocess_colnames_for_regression(outcomes_df)
outcomes_df

,outcome
subject_id,
1,-1.394524
2,1.352718
3,0.108480
4,-0.155989
6,0.348640
7,0.346888
9,-0.155989
11,0.352070
12,0.595661


Clinical Covariates

In [5]:
subject_column = 'ID'
clinical_information_column = 'Age'
#----------------------------------------------------------------DO NOT MODIFY--------------------------------------------------------
clinical_df_1 = pd.DataFrame()
clinical_df_1['clinical_data'] = data_df.loc[:, [clinical_information_column]]

clinical_df_1['subject_id'] = data_df.loc[:, [subject_column]]
# clinical_df_1['subject_id'] = [id.split('_')[0] for id in data_df[subject_column].to_list()] 
clinical_df_1.set_index('subject_id', inplace=True)

# Convert the 'subject_id' column to strings for each DataFrame
clinical_df_1.index = clinical_df_1.index.astype(str)
clinical_df_1 = preprocess_colnames_for_regression(clinical_df_1)
clinical_df_1

,clinical_data
subject_id,
1,57.0
2,50.0
3,62.0
4,50.0
6,60.0
7,73.0
9,64.0
11,62.0
12,54.0


Voxelwise Data

In [6]:
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
from calvin_utils.nifti_utils.generate_nifti import nifti_from_matrix
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers

#get conectivity values of interest
matrix_path = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STN_WURZBURG/vtasCompoundMNI152/subgroup_with_cognitive_scores/cog_decline_patient_connectivity'
matrix = import_matrices_from_folder(matrix_path, file_pattern='/*_T*.nii*')

# prepped_matrix = pd.DataFrame()
# for column in matrix.columns:
#     print(column)
#     new_name = int(column.split('_')[0]) #.split('T')[1]) #<----------------------------------THIS IS A BUG-CREATOR. MUST BE TAILORED TO PATIENT DATA

# #Set patients to those in the clinical data dataframe
matrix = matrix.transpose()
matrix['subject_id'] = [str(col).split('_')[0].split('MDST')[1] for col in matrix.index]
matrix.set_index('subject_id', inplace=True)
matrix.index = matrix.index.astype(str)
neuroimaging_df_1 = preprocess_colnames_for_regression(matrix)
neuroimaging_df_1

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STN_WURZBURG/vtasCompoundMNI152/subgroup_with_cognitive_scores/cog_decline_patient_connectivity/*_T*.nii*
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split name
cannot further split nam

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_902619,var_902620,var_902621,var_902622,var_902623,var_902624,var_902625,var_902626,var_902627,var_902628
subject_id,,,,,,,,,,,,,,,,,,,,,
43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Voxelwise Data 2

In [7]:
#Import Another Dataframe if desired
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
from calvin_utils.nifti_utils.generate_nifti import nifti_from_matrix
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers

#get conectivity values of interest
matrix_path = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/cat12/cat12_ultrafine-reg/CAT12.8.2_2170'
neuroimaging_df_2 = import_matrices_from_folder(matrix_path, file_pattern='/*/*/*/*mwp1*resamp*.nii*')
performed_z_score = False 
#----------------------------------------------------------------DO NOT MODIFY!----------------------------------------------------------------

#Set patients to those in the clinical data dataframe
# prepped_matrix = prepped_matrix.loc[:, data_df.index]

neuroimaging_df_2 = neuroimaging_df_2.transpose()
neuroimaging_df_2['subject_id'] = [col.split('_')[0] for col in neuroimaging_df_2.index]
neuroimaging_df_2.set_index('subject_id', inplace=True)

neuroimaging_df_2.index = neuroimaging_df_2.index.astype(str)


#Display results
neuroimaging_df_2 = preprocess_colnames_for_regression(neuroimaging_df_2)
display(neuroimaging_df_2)

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/cat12/cat12_ultrafine-reg/CAT12.8.2_2170/*/*/*/*mwp1*resamp*.nii*


,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_902619,var_902620,var_902621,var_902622,var_902623,var_902624,var_902625,var_902626,var_902627,var_902628
subject_id,,,,,,,,,,,,,,,,,,,,,
130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Masking

In [8]:
#Mask Dataframes
mask_path = None # '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/voxelwise_glm/stim_by_age/palm_results/palm_statistic_topology/generated_nifti.nii' #None #
masking_df = None # # 
#----------------------------------------------------------------DO NOT MODIFY
from calvin_utils.nifti_utils.matrix_utilities import mask_matrix
# def mask_matrix(df_1, mask_path=None, mask_threshold=0.2, mask_by='rows', dataframe_to_mask_by=None):

neuroimaging_df_1 = mask_matrix(neuroimaging_df_1, mask_path=mask_path, mask_threshold=0, mask_by='columns', dataframe_to_mask_by=masking_df)
# neuroimaging_df_2 = mask_matrix(neuroimaging_df_2, mask_path=mask_path, mask_threshold=0, mask_by='columns', dataframe_to_mask_by=masking_df)

Dataframes have been masked such that their shapes are:  (32, 225222)


Optional - Ensure Aligned Dataframes

In [9]:
# Suppose df1, df2, df3 are your dataframes
common_index = outcomes_df.index.intersection(clinical_df_1.index).intersection(neuroimaging_df_1.index)

outcomes_df = outcomes_df.loc[common_index]
clinical_df_1 = clinical_df_1.loc[common_index]
neuroimaging_df_1 = neuroimaging_df_1.loc[common_index]
outcomes_df

,outcome
subject_id,
11,0.352070
12,0.595661
14,0.840954
15,-2.083833
17,0.595661
18,0.848013
19,0.108480
21,-2.862636
22,0.595661


# Save Dataframes for Access by Script
    """
    Saves DataFrames to CSV files and returns the paths.

    Parameters:
    - outcome_dfs (list): A list of outcome DataFrames.
    - covariate_dfs (list): A list of covariate DataFrames.
    - voxelwise_dfs (list): A list of voxelwise DataFrames.
    - path_to_dataframes (str): The directory where the DataFrames will be saved.

    Returns:
    - dict: A dictionary containing lists of paths for each DataFrame type.
    """

In [10]:
from calvin_utils.file_utils.dataframe_utilities import save_dataframes_to_csv

where_to_save = "/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STND_WURZBURG/response_topology/f_test/server_prep"
#----------------------------------------------------------------
df_paths_dict = save_dataframes_to_csv(outcome_dfs = [outcomes_df], 
                                       covariate_dfs = [clinical_df_1],
                                       voxelwise_dfs = [neuroimaging_df_1], 
                                       path_to_dataframes =where_to_save)
print(df_paths_dict)

{'outcomes': ['/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STND_WURZBURG/response_topology/f_test/server_prep/outcome_data_1.csv'], 'covariates': ['/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STND_WURZBURG/response_topology/f_test/server_prep/covariate_data_1.csv'], 'voxelwise': ['/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STND_WURZBURG/response_topology/f_test/server_prep/voxelwise_data_1.csv']}


# Submit to cluster


**Display Statistical Methods You Can Use**

In [11]:
from calvin_utils.file_utils.script_printer import ScriptInfo
from calvin_utils.permutation_analysis_utils.scripts_for_submission.script_descriptions import script_dict
#----------------------------------------------------------------
info = ScriptInfo(script_dict)
info.print_high_level_info()

Script: launch_f_test_palm.py
Method: F-Test
---


**Choose Method**

Copy the method you want to run fromt the output above into the cell below


In [12]:
method_to_use = 'F-Test'

**Transfer Files To Server**

This is going to upload the data and the script you will use.
___

    ScpTransfer class provides a convenient way to transfer files to a remote server
    using SCP (Secure Copy).
    Will request your password for transfer if no ssh_key provided.

    Example:
        remote_base_path = "/path/on/remote/server"
        remote_hostname = "example.com"
        remote_username = "your_username"
        ssh_key = "path/to/ssh_key/"

        scp_transfer = ScpTransfer(remote_hostname, remote_username, remote_password)
        scp_transfer.transfer_files_in_dict(dict_files_to_transfer, remote_base_path)

In [20]:
import importlib
from calvin_utils.server_utils.file_transfer_helper import ScpTransfer
remote_path_to_save_to = '/PHShome/cu135/permutation_tests/f_test/age_by_stim_pd_dbs_redone/inputs'
server = "erisone.partners.org"
username="cu135"
#----------------------------------------------------------------
df_paths_dict['python_script'] = [importlib.import_module(info.get_module_by_method(method_to_use)).__file__]
scp_transfer = ScpTransfer(hostname=server,
                           username=username, 
                           ssh_key=None
                        )
remote_df_paths_dict = scp_transfer.transfer_files_in_dict(dict_files=df_paths_dict, 
                                    base_remote_path=remote_path_to_save_to)

gaierror: [Errno 8] nodename nor servname provided, or not known

**Print Input Arguments and Dictionary for Script**

In [14]:
#----------------------------------------------------------------
inputs = info.get_inputs_by_method(method_to_use)
inputs

{'n_cores': 'The number of cores per job submission cpu (4 is a good default).',
 'out_dir': 'The output directory where the result csv files will be saved.',
 'job_name': 'The job name for identification.',
 'memory_per_job': 'The memory (in gigabytes) per job submission cpu. General max is 40 up to 498 Gb.',
 'outcome_data_path': 'The path to the outcome data csv file.',
 'clinical_covariate_paths': 'The paths to the clinical covariate data csv files.',
 'neuroimaging_df_paths': 'The paths to the voxelwise neuroimaging data csv files.'}

**Enter Arguments for a Script**

Copy the dictionary printed above into the cell below and fill it out. 

For example
 
 script_inputs_dict =
 
  {
    
 'n_cores': 16,

 'out_dir': "/PHShome/cu135/permutation_tests/f_test/age_by_stim_ad_dbs_redone/results/tmp",

 'job_name': 'ftest_bm',

 'outcome_data_path': remote_df_paths_dict['outcomes'],

 'clinical_covariate_paths': remote_df_paths_dict['covariates'],

 'voxelwise_data_paths': remote_df_paths_dict['voxelwise']
 
 }

In [25]:
script_inputs_dict = {
 'n_cores': 5,
 'out_dir': f"{remote_path_to_save_to}/results/raw_results",
 'job_name': 'f_test_pd',
 'memory_per_job': 8,
 'outcome_data_path': remote_df_paths_dict['outcomes'],
 'clinical_covariate_paths': remote_df_paths_dict['covariates'],
 'neuroimaging_df_paths': remote_df_paths_dict['voxelwise']
 }

**Submit Jobs**

    Initialize server
    server = LSFServer("erisone.partners.org", "cu135")

    Initialize job
    job = LSFJob(
    job_name="f_test_bm",
    user_email="choward12@bwh.harvard.edu",
    cpus=1,
    output_dir="/PHShome/cu135/terminal_outputs",
    error_dir="/PHShome/cu135/error_outputs",
    queue="big-multi",
    script_path="/PHShome/cu135/python_scripts/launch_f_test_palm.py",
    options="-o /PHShome/cu135/permutation_tests/f_test/age_by_stim_pd_dbs/results",
    Gb_requested=4,
    wait_time=0
    )

    Initialize job submitter
    job_submitter = JobSubmitter(server, job)

    Submit jobs
    job_submitter.submit_jobs(10)

This example will log into the server 'erisone.partners.org' as user 'cu135'.
 
Will submit 10 jobs named 'f_test_bm' to the queue 'big-multi'. 

Each job will run the Python script at '/PHShome/cu135/python_scripts/launch_f_test_palm.py' with the options specified.

Putputting to '/PHShome/cu135/terminal_outputs' and '/PHShome/cu135/error_outputs' for terminal and error logs respectively. 

The job requests 1 CPU and 4GB of memory.


For a guide to job submission:

https://rc.partners.org/kb/article/1462

In [28]:
from calvin_utils.server_utils.job_submission_helper import LSFServer, LSFJob, JobSubmitter
# Create an instance of the LSF job.
# Assuming a resource requirement of 500GB memory and 6 cores for each job.
# And a job script with some options.
server_name = "cn050.research.partners.org"
user_email = "choward12@bwh.harvard.edu"
user_name = "cu135"
num_permutations = 10000
queue_name = "normal"
server_env_activation_string = "conda activate nimlab"
#----------------------------------------------------------------
lsf_job = LSFJob(job_name=script_inputs_dict['job_name'],
                 user_email=user_email,
                 output_dir="~/terminal_outputs",
                 error_dir="~/terminal_outputs",
                 queue=queue_name,
                 n_jobs=int(np.round(num_permutations/script_inputs_dict['n_cores'])),
                 cpus=script_inputs_dict['n_cores'],
                 gb_requested=script_inputs_dict['memory_per_job'],
                 wait_time=None,
                 script_path=remote_df_paths_dict['python_script'][0],
                 environment_activation_string=server_env_activation_string,
                 options=script_inputs_dict
                 )

lsf_server = LSFServer(server_name=server_name, 
                       username=user_name)

job_submitter = JobSubmitter(lsf_server, lsf_job)
job_command = job_submitter.submit_jobs(print_job=True)

Failed to submit jobs due to error: [Errno 8] nodename nor servname provided, or not known
Job submitted successfully:  False


UnboundLocalError: local variable 'job_command' referenced before assignment

In [29]:
print(job_command)

bsub -q bigmem -n 5 -R 'rusage[mem=8000] span[ptile=5]' -M 8000 -o ~/terminal_outputs/f_test_pd_%I.txt -J 'f_test_pd[1-2000]' -u choward12@bwh.harvard.edu -cwd /PHShome/cu135/permutation_tests/f_test/age_by_stim_pd_dbs_redone/inputs/python_script 'source ~/.bashrc && conda activate nimlab &&  python /PHShome/cu135/permutation_tests/f_test/age_by_stim_pd_dbs_redone/inputs/python_script/launch_f_test_palm.py --n_cores "5" --out_dir "/PHShome/cu135/permutation_tests/f_test/age_by_stim_pd_dbs_redone/inputs/results/raw_results" --job_name "f_test_pd" --memory_per_job "8" --outcome_data_path [\""/PHShome/cu135/permutation_tests/f_test/age_by_stim_pd_dbs_redone/inputs/outcomes/outcome_data_1.csv"\"] --clinical_covariate_paths [\""/PHShome/cu135/permutation_tests/f_test/age_by_stim_pd_dbs_redone/inputs/covariates/covariate_data_1.csv"\"] --neuroimaging_df_paths [\""/PHShome/cu135/permutation_tests/f_test/age_by_stim_pd_dbs_redone/inputs/voxelwise/voxelwise_data_1.csv"\"]'


# Recompose Data
This code uses a file-staging approach to large-scale computation. The resultant files have been saved to your output directory. You must now recompose them. 

In [ ]:

import os
import pandas as pd

def combine_csvs(directory, output_filename):
    """
    Combine all CSV files in a directory into a single CSV file.

    Parameters:
    - directory (str): The path to the directory containing the CSV files.
    - output_filename (str): The path to the output CSV file.
    """
    # Initialize an empty DataFrame
    combined_df = pd.DataFrame()

    # Get a list of all CSV files in the directory
    csv_files = [f for f in os.listdir(directory) if f.endswith(".csv")]

    # Loop through the CSV files and append each one to the combined DataFrame
    for csv_file in csv_files:
        df = pd.read_csv(os.path.join(directory, csv_file))
        combined_df = pd.concat([combined_df, df])

    # Save the combined DataFrame as a new CSV file
    combined_df.to_csv(output_filename, index=False)
    return output_filename


In [ ]:
recomposed_csv_path = combine_csvs("/path/to/your/directory", "combined.csv")
recomposed_csv_df = pd.read_csv(recomposed_csv_path)
recomposed_csv_df

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/your/directory'

In [ ]:
#import confidence_intervals
# run confidence intervals on the csv

import multiprocessing as mp
import numpy as np
import pandas as pd
from tqdm import tqdm
from statsmodels.formula.api import ols

def calculate_confidence_intervals(ab_paths, mediators):
    """
    Calculates the confidence intervals and p-value based on the bootstrapped samples.

    Parameters:
    - ab_paths: list of lists containing the bootstrapped ab paths for each mediator.
    - total_indirect_effects: list of bootstrapped summed ab paths.
    - mediators: list of mediator names.

    Returns:
    - DataFrame with the mean indirect effect, confidence intervals, and p-values for each mediator and the total indirect effect.
    """
    ab_path_values = np.array(ab_paths)

    # Check if there's only one mediator
    if isinstance(mediators, str):
        mediators = [mediators]

    # Calculate mean indirect effect and confidence intervals for each mediator
    mean_ab_paths = np.mean(ab_path_values, axis=0)
    lower_bounds = np.percentile(ab_path_values, 2.5, axis=0)
    upper_bounds = np.percentile(ab_path_values, 97.5, axis=0)

    # Calculate p-values for each mediator
    ab_path_p_values = [np.mean(np.sign(mean_ab_paths) * ab_path_values <= 0)]

    # Create DataFrame to store the results
    result_df = pd.DataFrame({
        'Point Estimate': mean_ab_paths,
        '2.5th Percentile': lower_bounds,
        '97.5th Percentile': upper_bounds,
        'P-value': ab_path_p_values
    }, index=mediators)

    return result_df


Get Results

In [ ]:
pool = mp.Pool(mp.cpu_count())
results = list(tqdm(pool.imap(calculate_confidence_intervals, [row for _, row in recomposed_csv_df.iterrows()]), total=recomposed_csv_df.shape[0]))
pool.close() 

# Create the results DataFrame
results_df = pd.concat(results, axis=1).transpose()

# Create Nifti Results

In [ ]:
# results_df = pd.read_csv('/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/mediated_moderation/age_mediated_by_grey_matter/P-value_mediated_moderation_analysis_results.csv')

mask_threshold=0
#----------------------------------------------------------------DO NOT TOUCH
#Prepare Indices to Unmask
from calvin_utils.nifti_utils.matrix_utilities import unmask_matrix
from nimlab import datasets as nimds
from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti
from nilearn import image

#Perform Unmasking
if mask_path is not None:
    mni_mask = image.load_img(mask_path).get_fdata().flatten()
    brain_indices = np.where(mni_mask > mask_threshold)[0]
elif masking_df is not None:
    mni_mask = nimds.get_img("mni_icbm152").get_fdata().flatten()
    mask = masking_df.transpose().reset_index(drop=True).copy()
    mask['mask_index'] = mask.sum(axis=1)
    brain_indices = np.where(mask['mask_index'] != 0)[0]
else:
    mni_mask = nimds.get_img("mni_icbm152").get_fdata().flatten()
    brain_indices = np.where(mni_mask > 0)[0]

# Create a boolean mask for brain_indices
bool_mask = np.zeros_like(mni_mask, dtype=bool)
bool_mask[brain_indices] = True

for statistic in results_df.columns:
    try:
        print(statistic)
        
        # Initialize the output mask with NaN values
        output_mask = np.full_like(mni_mask, np.nan)
        
        # Reinstate the values at brain_indices
        output_mask[bool_mask] = results_df[statistic]
        
        # View and save
        view_and_save_nifti(output_mask, out_dir=out_dir, output_name=statistic)
    except:
        print(f"Couldn't convert {statistic} to nifti")

Done